<a href="https://colab.research.google.com/github/HAR5HA-7663/MCS-5993-Evolutionary-Computation-and-Deep-Learning/blob/main/Assignments/4/XOR3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## XOR with 3 classes

<pre>
   | x0 | x1 |XOR3|
   |----|----|----|
   |0.0 |0.0 | 0  |
   |0.0 |1.0 | 1  |
   |1.0 |0.0 | 1  |
   |1.0 |1.0 | 0  |
   |0.5 |0.5 | 2  |
</pre>

In [28]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

# the five different states of the XOR3 gate
X = np.array([[0,0],[0,1],[1,0],[1,1],[0.5,0.5],[0.6,0.6]])

# the five expected results in the same order
y = np.array([[0],[1],[1],[0],[2],[2]])

Xt = np.array([[0,0.1],[0.1,1],[0.9,0],[0.9,1],[0.55,0.45]])
yt = np.array([[0],[1],[1],[0],[2]])

In [29]:
act_func = ('relu', 'elu', 'sigmoid', 'tanh', 'leaky_relu')
optimz = ('SGD', 'RMSprop', 'Adam')

print("Activation functions:", act_func)
print("Optimizers:", optimz)

Activation functions: ('relu', 'elu', 'sigmoid', 'tanh', 'leaky_relu')
Optimizers: ('SGD', 'RMSprop', 'Adam')


In [30]:
# Using 'sparse_categorical_crossentropy'
def XOR3_eval(param):
    """
    param indices:
      0: num of hidden neurons
      1: learning rate
      2: batch size
      3: activation function index
      4: loss function index (fixed, not really used)
      5: optimizer index
    """
    global model
    model = Sequential([
        keras.Input(shape=(2,)),
        Dense(int(param[0]), input_dim=2, activation=act_func[round(param[3])]),
        Dense(3, activation='softmax')
    ])

    if round(param[5]) == 0:
        optmzr = optimizers.SGD(learning_rate=param[1])
    elif round(param[5]) == 1:
        optmzr = optimizers.RMSprop(learning_rate=param[1])
    else:
        optmzr = optimizers.Adam(learning_rate=param[1])

    model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy', # target labels are integers
    metrics=['accuracy']
    )
    model.fit(X, y, batch_size=1, epochs=1300, verbose=0)

    loss, acc = model.evaluate(Xt, yt, verbose=0)
    return loss

# test_param = [4, 0.5, 2, 2, 0, 1]  # neurons, lr, bsize, act='sigmoid', loss, opt='RMSprop'
# print("loss =", XOR3_eval(test_param))

In [31]:
def print_HP_found(x, eval):
    print(
        f"#neurons={int(x[0])}, "
        f"lr={x[1]:.3f}, "
        f"bsize={int(x[2])}, "
        f"actF={act_func[int(round(x[3]))]}, "
        f"lossF='{loss_func}', "
        f"optim={optimz[int(round(x[5]))]},  "
        f"Eval={eval:.6f}"
    )

In [ ]:
import numpy as np
from keras.models import save_model

# constants
MaxGen   = 35    # generations per trial
MaxTrial = 3     # number of independent trials
numVar   = 6     # number of hyperparameters

# initial step size and adjustment ratio
stepSize_i = 0.82     # initial step size
stepSize_r = 0.82     # step size ratio (Rechenberg constant)

totgen  = 0
foundCnt = 0
objfunc = XOR3_eval   # link to objective function

print("[#neurons, lr, bsize, actfun, lossfun, optmzr]")

for trial in range(MaxTrial):
    print(f"\n************************** Trial # = {trial+1}")
    best_p_val = 9999.0
    xp = np.empty(numVar)  # parent
    xo = np.empty(numVar)  # offspring
    successCnt = 0
    WindowSize = 10
    stepSize = stepSize_i

    # --- initialize hyperparameters (start point) ---
    hparams = [
        (np.random.randint(2, 16), 2, 16),          # hidden neurons
        (round(np.random.uniform(0.01, 1.5), 2), 0.01, 1.5),  # learning rate
        (np.random.randint(1, 5), 1, 5),            # batch size
        (np.random.randint(0, len(act_func)), 0, len(act_func)-1),  # activation
        (0, 0, 0),                                  # loss fixed
        (np.random.randint(0, len(optimz)), 0, len(optimz)-1)       # optimizer
    ]

    for i in range(numVar):
        xp[i] = hparams[i][0]

    # evaluate parent
    p_val = objfunc(xp)

    # --- evolution loop ---
    for g in range(1, MaxGen + 1):

        # 1/5 success rule step-size adaptation
        if (g % WindowSize) == 0:
            if successCnt > (WindowSize * 0.2):
                stepSize /= stepSize_r
            elif successCnt < (WindowSize * 0.2):
                stepSize *= stepSize_r
            successCnt = 0

        # mutate each hyperparameter with scaled step size
        for i in range(numVar):
            range_scale = hparams[i][2] - hparams[i][1]
            variation = np.random.normal(0.0, stepSize) * range_scale
            if i == 1:   # learning rate: keep decimals
                xo[i] = round(xp[i] + variation, 2)
            else:
                xo[i] = xp[i] + variation
            xo[i] = np.clip(xo[i], hparams[i][1], hparams[i][2])

        # evaluate offspring
        o_val = objfunc(xo)

        # selection
        if o_val < p_val:
            xp = xo.copy()
            p_val = o_val
            successCnt += 1

        # check acceptable solution
        if p_val < 0.01:
            print(f"Acceptable solution found after {g} generations:")
            print_HP_found(xp, p_val)
            if p_val < best_p_val:
                best_p_val = p_val
                save_model(model, "bestXOR.keras")
            totgen += g
            foundCnt += 1
            break  # stop this trial early if success

    # end of one trial
print(f"\nSystem Success = {foundCnt / MaxTrial * 100:.2f}%")
totgen += (MaxTrial - foundCnt) * MaxGen
print(f"Average # of generations used = {totgen / MaxTrial:.0f}")

[#neurons, lr, bsize, actfun, lossfun, optmzr]

************************** Trial # = 1


In [26]:
print (model.predict(X))
print (model.predict(X, verbose=0).round())
(loss, acc) = model.evaluate(Xt, yt)
print (f"loss = {loss}, acc = {acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.32891116 0.39138672 0.27970216]
 [0.32705677 0.33948353 0.33345968]
 [0.3399697  0.32574695 0.33428335]
 [0.33203685 0.2821782  0.38578492]
 [0.33333313 0.33189288 0.3347739 ]
 [0.33345464 0.32090342 0.34564197]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2000 - loss: 1.1023
loss = 1.1023192405700684, acc = 0.20000000298023224


In [4]:
# model save into a file and reloading the model
model.summary()
model.save('xor3.keras') # the file is stored on the cloud account. Will stay a day

model.load_weights('xor3.keras')
print (model.predict(X))
print (model.predict(X, verbose=0).round())
model.evaluate(Xt, yt)

m2 = keras.models.load_model('xor3.keras')
m2.summary()
print (m2.predict(X))
print (m2.predict(X, verbose=0).round())
(loss, acc) = m2.evaluate(Xt, yt)
print (f"loss = {loss}, acc = {acc}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200 (804.00 B)

 Trainable params: 99 (396.00 B)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 200 (804.00 B)

 Trainable params: 99 (396.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 101 (408.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[9.7825152e-01 4.9585537e-03 1.6790016e-02]
 [1.2876927e-04 9.9973458e-01 1.3668477e-04]
 [1.0027302e-03 9.9895966e-01 3.7572110e-05]
 [6.3272476e-01 3.6554886e-03 3.6361971e-01]
 [5.4799873e-01 7.2234357e-03 4.4477791e-01]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.8000 - loss: 0.3109
loss = 0.3108638823032379, acc = 0.800000011920929


In [5]:
# Using 'categorical_crossentropy'

model = Sequential([
    keras.Input(shape=(2,)),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(
  optimizer='rmsprop',
  loss='categorical_crossentropy', # target labels are 1hot encoded format
  metrics=['accuracy']
)

# Converting y to one-hot encoded format with shape (5,3)
y_1hot = keras.utils.to_categorical(y, num_classes=3)
model.fit(X, y_1hot, batch_size=1, epochs=1300, verbose=0)
print (model.predict(X))
print (model.predict(X, verbose=0).round())
yt_1hot = keras.utils.to_categorical(yt, num_classes=3)
model.evaluate(Xt, yt_1hot)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[9.7105920e-01 2.6570898e-03 2.6283663e-02]
 [1.6168047e-04 9.9910325e-01 7.3514599e-04]
 [2.3601754e-03 9.9747491e-01 1.6491566e-04]
 [9.7372544e-01 6.9527287e-04 2.5579328e-02]
 [7.0759282e-02 2.5769309e-03 9.2666376e-01]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 0.0542


[0.05423687770962715, 1.0]